# Prep data for modeling by pulling current season in realtime
### NOTE:  this requires an updated history file, which must be manually refreshed each season)

In [ ]:
#Only run this code if you need to update the history file
from jobs import SRjobs
SRjobs.create_history()

### Run the next sections every week, or every time you add new feature engineering

1. Create past_games df from history and in-season scrape
2. Apply basic features (game_conf, SRTD, CRTD) to past_games df
3. Update the GL files (offense and defense)
4. Apply GL features to past_games df
5. Apply basic and GL features to future games

In [1]:
from jobs import SRjobs, SRfeatures, SRgamelogs
import pandas as pd
import importlib

# PAST GAMES
#scrape and/or get all the information from website/disk
schedule_mstr = SRjobs.get_season_played_games()
history = SRjobs.get_history()

#combine recent history with previous seasons, apply features
past_games = pd.concat([history, schedule_mstr])

Initiate get_season_played_games
Finished scraping 2018 schedule with 884 rows
Successfully loaded 845.0 played games from 2018


In [2]:
print('Applying features to games already played')
past_games = SRfeatures.apply_features(past_games)

Applying features to games already played
Added Game_conf
Added Team_SRTD
Added Team_CRTD
Added Opp_SRTD
Added Opp_CRTD


In [3]:
past_games.to_csv('/Users/markvonoven/Projects/CollegeFootball/SRoutput/past_games_w_features.csv')

In [4]:
past_games = pd.read_csv('/Users/markvonoven/Projects/CollegeFootball/SRoutput/past_games_w_features.csv')
past_games.set_index(['Season', 'Gamecode'], inplace=True)
past_games['Date'] = pd.to_datetime(past_games['Date'])
past_games = past_games.sort_values(by=['Date'])

Update GL files (O&D) here....

In [5]:
#Add GL raw stats for all played games
past_games_w_GLstats = SRgamelogs.add_GLstats(past_games)
past_games_w_GLstats.to_csv('/Users/markvonoven/Projects/CollegeFootball/SRoutput/past_games_w_GLstats.csv')

Updated Team_Off_sumstats
Updated Team_Off_meanstats
Updated Team_Def_sumstats
Updated Team_Def_meanstats
Finished updating the GLstats for the new past_games


In [6]:
#Add GL features using the raw stats you just created
from jobs import SRGLfeatures
past_games = SRGLfeatures.apply_features(past_games)

Added Team_OffStrat
Added Opp_OffStrat
Added Team_DefStren
Added Opp_DefStren


In [7]:
past_games.to_csv('/Users/markvonoven/Projects/CollegeFootball/SRoutput/past_games_w_SRGLfeatures.csv')

In [ ]:
#Add SOS ratings
from jobs import SRSOS
past_games['Team_SOS'] = past_games.apply(lambda x: SRSOS.strength_of_schedule(x['Team'], x['Date']), axis=1)
print('Added Team_SOS')
past_games['Opp_SOS'] = past_games.apply(lambda x: SRSOS.strength_of_schedule(x['Opp'], x['Date']), axis=1)
print('Added Opp_SOS')

In [ ]:
past_games.to_csv('/Users/markvonoven/Projects/CollegeFootball/SRoutput/past_games_w_SOS.csv')

In [10]:
# ignore this later!!!
past_games = pd.read_csv('/Users/markvonoven/Projects/CollegeFootball/SRoutput/past_games_w_SOS.csv')
past_games.set_index(['Season', 'Gamecode'], inplace=True)
past_games['Date'] = pd.to_datetime(past_games['Date'])
past_games = past_games.sort_values(by=['Date'])
from jobs import SRSOS

In [11]:
#scrape and/or get all the information from website/disk
future_games = SRjobs.get_season_future_games()

# Add the potential BCS championship matchups to the list of future games
date = pd.Timestamp(2019, 1, 7, 18)

df2 = pd.DataFrame([[2018, '201917AlaCle', date, 'Alabama', 'Clemson', 'TBD', 'Neutral', 0.96, 0.92],
                  [2018, '201917AlaNot', date, 'Alabama', 'Notre Dame', 'TBD', 'Neutral', 0.96, 0.88],
                  [2018, '201917CleOkl', date, 'Clemson', 'Oklahoma', 'TBD', 'Neutral', 0.92, 0.84],
                    [2018, '201917OklNot', date, 'Oklahoma', 'Notre Dame', 'TBD', 'Neutral', 0.92, 0.88]],
                  columns=['Season', 'Gamecode', 'Date', 'Team', 'Opp', 'Won', 'Game_home', 'Team_rank', 'Opp_rank'])

df2.set_index(['Season', 'Gamecode'], inplace=True)
future_games = future_games.append(df2, sort=False)

#load the full history, which is now past_games plus all the features
full_history = past_games_w_GLstats.copy()

#apply features to future games
print('Applying features to future games')
future_games = SRfeatures.apply_future_features(future_games, full_history)
future_games = SRGLfeatures.apply_features(future_games)
future_games['Team_SOS'] = future_games.apply(lambda x: SRSOS.strength_of_schedule(x['Team'], x['Date']), axis=1)
future_games['Opp_SOS'] = future_games.apply(lambda x: SRSOS.strength_of_schedule(x['Opp'], x['Date']), axis=1)

Initiate get_season_future_games
Finished scraping 2018 schedule with 884 rows
Successfully loaded future_mstr with 39 remaining games in 2018
Applying features to future games
Added Game_conf
Added Team_SRTD
Added Team_CRTD
Added Opp_SRTD
Added Opp_CRTD
Added Team_OffStrat
Added Opp_OffStrat
Added Team_DefStren
Added Opp_DefStren


In [ ]:
past_games.tail()

In [ ]:
future_games.tail(10)

### Normalize any features which require it

In [12]:
def nz_num(num, feature):
    min_DFS = min(past_games[feature])
    max_DFS = max(past_games[feature])
    return (num-min_DFS)/(max_DFS - min_DFS)

def nz_features(feature_list):
    for feature in feature_list:
        
        past_games[feature + '_nz'] = past_games.apply(lambda x: 
                                                      nz_num(x[feature], feature), axis=1)
        print('Normalized ' + feature + ' on past_games')
        future_games[feature + '_nz'] = future_games.apply(lambda x: 
                                                      nz_num(x[feature], feature), axis=1)
        print('Normalized ' + feature + ' on future_games')
        past_games.drop(feature, axis=1, inplace=True)
        future_games.drop(feature, axis=1, inplace=True)

feature_list = ['Team_OffStrat', 'Opp_OffStrat', 'Team_DefStren', 'Opp_DefStren']

nz_features(feature_list)

Normalized Team_OffStrat on past_games
Normalized Team_OffStrat on future_games
Normalized Opp_OffStrat on past_games
Normalized Opp_OffStrat on future_games
Normalized Team_DefStren on past_games
Normalized Team_DefStren on future_games
Normalized Opp_DefStren on past_games
Normalized Opp_DefStren on future_games


In [13]:
past_games = pd.get_dummies(past_games, prefix=['Game'], columns=['Game_home'])

In [14]:
future_games = pd.get_dummies(future_games, prefix=['Game'], columns=['Game_home'])

In [15]:
# just because bowl games have no home or away games
future_games['Game_Home'] = 0
future_games['Game_Away'] = 0

In [16]:
cols = ['Date', 'Team', 'Opp', 'Won', 'Team_rank', 'Opp_rank',
       'Game_conf', 'Team_SRTD', 'Team_CRTD', 'Opp_SRTD', 'Opp_CRTD',
       'Team_SOS', 'Opp_SOS', 'Team_OffStrat_nz', 'Opp_OffStrat_nz', 'Team_DefStren_nz',
       'Opp_DefStren_nz', 'Game_Away', 'Game_Home', 'Game_Neutral']
future_games = future_games[cols]

In [17]:
future_games

Date                   Team  \
Season Gamecode                                                    
2018   20181215LouTul 2018-12-15 13:30:00              Louisiana   
       20181215NorUta 2018-12-15 14:00:00            North Texas   
       20181215AriFre 2018-12-15 15:30:00          Arizona State   
       20181215EasGeo 2018-12-15 17:30:00       Eastern Michigan   
       20181215AppMid 2018-12-15 21:00:00      Appalachian State   
       20181218AlaNor 2018-12-18 19:00:00     Alabama-Birmingham   
       20181219OhiSan 2018-12-19 20:00:00                   Ohio   
       20181220MarSou 2018-12-20 20:00:00               Marshall   
       20181221FloTol 2018-12-21 12:30:00  Florida International   
       20181221BriWes 2018-12-21 16:00:00          Brigham Young   
       20181222MemWak 2018-12-22 12:00:00                Memphis   
       20181222ArmHou 2018-12-22 15:30:00                   Army   
       20181222BufTro 2018-12-22 19:00:00                Buffalo   
       20181222HawLou 2018-12-22 22:30:00                 Hawaii   
       20181226BoiBos 2018-12-26 13:30:00            Boise State   
       20181226GeoMin 2018-12-26 17:15:00           Georgia Tech   
       20181226CalTex 2018-12-26 21:00:00             California   
       20181227DukTem 2018-12-27 13:30:00                   Duke   
       20181227MiaWis 2018-12-27 17:15:00             Miami (FL)   
       20181227BayVan 2018-12-27 21:00:00                 Baylor   
       20181228AubPur 2018-12-28 13:30:00                 Auburn   
       20181228SyrWes 2018-12-28 17:15:00               Syracuse   
       20181228IowWas 2018-12-28 21:00:00             Iowa State   
       20181229FloMic 2018-12-29 12:00:00                Florida   
       20181229SouVir 2018-12-29 12:00:00         South Carolina   
       20181229ArkNev 2018-12-29 13:15:00         Arkansas State   
       20181229CleNot 2018-12-29 16:00:00                Clemson   
       20181229AlaOkl 2018-12-29 20:00:00                Alabama   
       20181231CinVir 2018-12-31 12:00:00             Cincinnati   
       20181231PitSta 2018-12-31 14:00:00             Pittsburgh   
       20181231MicOre 2018-12-31 15:00:00         Michigan State   
       20181231MisOkl 2018-12-31 15:45:00               Missouri   
       20181231NorUta 2018-12-31 19:00:00           Northwestern   
       20181231NorTex 2018-12-31 19:30:00   North Carolina State   
       201911IowMis   2019-01-01 12:00:00                   Iowa   
       201911KenPen   2019-01-01 13:00:00               Kentucky   
       201911CenLou   2019-01-01 13:00:00        Central Florida   
       201911OhiWas   2019-01-01 17:00:00             Ohio State   
       201911GeoTex   2019-01-01 20:45:00                Georgia   
       201917AlaCle   2019-01-07 18:00:00                Alabama   
       201917AlaNot   2019-01-07 18:00:00                Alabama   
       201917CleOkl   2019-01-07 18:00:00                Clemson   
       201917OklNot   2019-01-07 18:00:00               Oklahoma   

                                          Opp  Won  Team_rank  Opp_rank  \
Season Gamecode                                                           
2018   20181215LouTul                  Tulane  TBD       0.00      0.00   
       20181215NorUta              Utah State  TBD       0.00      0.00   
       20181215AriFre            Fresno State  TBD       0.00      0.24   
       20181215EasGeo        Georgia Southern  TBD       0.00      0.00   
       20181215AppMid  Middle Tennessee State  TBD       0.00      0.00   
       20181218AlaNor       Northern Illinois  TBD       0.00      0.00   
       20181219OhiSan         San Diego State  TBD       0.00      0.00   
       20181220MarSou           South Florida  TBD       0.00      0.00   
       20181221FloTol                  Toledo  TBD       0.00      0.00   
       20181221BriWes        Western Michigan  TBD       0.00      0.00   
       20181222MemWak             Wake Forest  TBD       0.00      0.00   
 

In [18]:
past_games.to_csv('./SRinput/current/past_games.csv')

In [19]:
future_games.to_csv('./SRinput/current/future_games.csv')